In [1]:
import pandas as pd
import numpy as np

#### Get game stats from CFBD API.

In [ ]:
import time
import cfbd
from cfbd.rest import ApiException
from dotenv import dotenv_values

# Configure API key authorization: ApiKeyAuth
env_vars = dotenv_values('.env')

configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = 'zbg8Bi/2L7I3THKreAd+Z1MN3BQAQ7QJy5Csc10X8ByaOzAyWK+qsXC59U6Z96ap'
configuration.api_key_prefix['Authorization'] = 'Bearer'

api_config = cfbd.ApiClient(configuration)
api_instance = cfbd.GamesApi(api_config)

def get_game_info(games) :
    team_dict = [
        dict(id = g.id,
            team1 = g.teams[0].school,
            team1_pts = g.teams[0].points,
            team1_home_away = g.teams[0].home_away,
            **{"team1_" + x.category : x.stat for x in g.teams[0].stats},
            team2 = g.teams[1].school,
            team2_pts = g.teams[1].points,
            team2_home_away = g.teams[1].home_away,
            **{"team2_" + x.category : x.stat for x in g.teams[1].stats}) for g in games]
    return pd.DataFrame(team_dict)

def get_games_for_year(api, year) :
    data_frames = []
    for week in range(1, 17) :
        print("Fetching week " + str(week) + ".\n")
        api_response = api.get_team_game_stats(year=year, week=week)
        game_stats = get_game_info(api_response)
        data_frames.append(game_stats)
        time.sleep(1)
    return pd.concat(data_frames, ignore_index=True)

In [ ]:
for year in range(2024, 2001, -1) :
    api_instance = cfbd.GamesApi(api_config)
    df = get_games_for_year(api_instance, year)
    df.to_csv("game_by_game_stats/games_" + str(year) + ".csv")
    time.sleep(1)

#### Gather game ids and dates.

In [ ]:
def get_game_id_date(games) :
    date_dict = [
        dict(id = g.id,
            date = g.start_date,
            home_team = g.home_team,
            away_team = g.away_team,
            home_pregame_elo = g.home_pregame_elo,
            away_pregame_elo = g.away_pregame_elo,
            conference_game = g.conference_game,
            season_type = g.season_type,
            home_division = g.home_division,
            away_division = g.away_division)
    for g in games]
    
    return pd.DataFrame(date_dict)

def get_game_dates_for_year(api, year) :
    data_frames = []
    for week in range(1,17) :
        print("Fetching week " + str(week) + ".\n")
        api_response = api.get_games(year=year, week=week)
        game_dates = get_game_id_date(api_response)
        data_frames.append(game_dates)
    return pd.concat(data_frames, ignore_index=True)

In [ ]:
for year in range(2024, 2001, -1) :
    df = get_game_dates_for_year(api_instance, year)
    df.to_csv("game_by_game_stats/game_dates" + str(year) + ".csv")